Q. 아래 사항들을 참조하여 타이타닉 생존자 예측모델 분석용 데이터셋을 구축 및 모델링에 적용한 결과를 출력하세요.

- 분석 데이터 : train.csv

[Titanic data 전처리 내역]  

- 불필요한 속성 칼럼 삭제
- Null 값 처리
- 속성 변환(범주화 등)
- 문자열 칼럼 레이블 또는 원핫인코딩 
- 재사용 가능한 전처리 사용자 함수 작성 하여 일괄 전처리

[탐색적 분석 필수 항목]
- 통계적 시각적 변수 탐색
- 파생변수 : 분석에 필요한 파생변수 1개 이상 생성
- 탐색 결과에 대한 종합 의견

[분석용 데이터셋 구축]
- 제시된 모델링 방법 및 평가 방법 활용, 분석용 데이터셋 적용 및 정확도 산출

※ 컬럼 정보  
- PassengerId : 승객 번호
- Survived : 생존여부(1: 생존, 0 : 사망)
- Pclass : 승선권 클래스(1 : 1st, 2 : 2nd ,3 : 3rd)
- Name : 승객 이름
- Sex : 승객 성별
- Age : 승객 나이 
- SibSp : 동반한 형제자매, 배우자 수
- Patch : 동반한 부모, 자식 수
- Ticket : 티켓의 고유 넘버
- Fare 티켓의 요금
- Cabin : 객실 번호
- Embarked : 승선한 항구명(C : Cherbourg, Q : Queenstown, S : Southampton)

In [518]:
import pandas as pd
import numpy as np
tdf = pd.read_csv('dataset/train.csv')
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [512]:
tdf['Embarked'].fillna(method='ffill',inplace=True) # 승선한 항구명의 Null값은 바로 앞에 있는 값으로 채움
tdf.drop('Cabin',axis=1,inplace=True) # 관련 없는 컬럼 삭제
tdf.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         0
dtype: int64

In [513]:
# Age의 NaN 값을 어떻게 채울지(class,fare(0 = 관계자 or 무임승차) 높을 수록 나이가 많음, 배우자가 있는 경우의 나이, 형제자매가 있는 동시에 부모,자식이 있다 = 자녀일 가능성 높음)
tdf['Age'].fillna(method='ffill',inplace=True)

# 티켓 고유넘버 분석하기
# 객실 고유넘버랑 티켓 고유넘버 연관지어서 보기
# 탑승 항구별 데이터 확인

In [514]:
# 범주화 및 라벨링
bins = [0,15,25,45,65,100]
labels = ['어린이','청년','장년','중년','노년']
tdf['Age_text'] = pd.cut(x=tdf['Age'],bins=bins,labels=labels,include_lowest=True) #include_lowest = 가장 작은 값을 포함
tdf['Age'].dropna()


0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888    19.0
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [515]:
dt = pd.get_dummies(tdf['Sex'])
dt1 = pd.get_dummies(tdf['Embarked'],prefix='Embarked')
og = pd.concat([dt,dt1],axis=1)
tdf = pd.concat([tdf,og],axis=1)
tdf.drop(['Sex','Embarked','Ticket','Name','Age_text'],axis=1,inplace=True)
tdf

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000,0,1,0,0,1
887,888,1,1,19.0,0,0,30.0000,1,0,0,0,1
888,889,0,3,19.0,1,2,23.4500,1,0,0,0,1
889,890,1,1,26.0,0,0,30.0000,0,1,1,0,0


In [516]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
y = tdf['Survived']
X = tdf.drop('Survived',axis=1)
# 학습용 테이터와 평가용 데이터를 8:2로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10) 

In [517]:
# Random Forest
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_pred)
print('rf 예측 정확도 :', rf_accuracy)

rf 예측 정확도 : 0.8603351955307262
